In [107]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import shapiro


In [77]:
from google.colab import files
uploaded = files.upload()

Saving Nasa.csv to Nasa (1).csv


In [78]:
data = pd.read_csv('Nasa.csv')
data

,Frequency,AngleOfAttack,ChordLength,FreeStreamVelocity,SuctionSideDisplacementThickness,ScaledSoundPressureLevel
0,800,0.0,3.048,71.3,266.337,126.201
1,1000,0.0,3.048,71.3,266.337,125.201
2,1250,0.0,3.048,71.3,266.337,125.951
3,1600,0.0,3.048,71.3,266.337,127.591
4,2000,0.0,3.048,71.3,266.337,127.461
...,...,...,...,...,...,...
1498,2500,15.6,1.016,39.6,528.487,110.264
1499,3150,15.6,1.016,39.6,528.487,109.254
1500,4000,15.6,1.016,39.6,528.487,106.604
1501,5000,15.6,1.016,39.6,528.487,106.224


In [96]:
#Análisis de datos exploratorio
data.head()
data.info()
data.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503 entries, 0 to 1502
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Frequency                         1503 non-null   int64  
 1   AngleOfAttack                     1503 non-null   float64
 2   ChordLength                       1503 non-null   float64
 3   FreeStreamVelocity                1503 non-null   float64
 4   SuctionSideDisplacementThickness  1503 non-null   float64
 5   ScaledSoundPressureLevel          1503 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 70.6 KB


,Frequency,AngleOfAttack,ChordLength,FreeStreamVelocity,SuctionSideDisplacementThickness,ScaledSoundPressureLevel
count,1503.000000,1503.000000,1503.000000,1503.000000,1503.000000,1503.000000
mean,2886.380572,6.782302,128.322896,50.860745,333.091439,124.835943
std,3152.573137,5.918128,190.225714,15.572784,219.899327,6.898657
min,200.000000,0.000000,1.016000,31.700000,2.211000,103.380000
25%,800.000000,2.000000,1.524000,39.600000,152.689000,120.191000
50%,1600.000000,5.400000,2.286000,39.600000,293.031000,125.721000
75%,4000.000000,9.900000,254.000000,71.300000,461.377000,129.995500
max,20000.000000,22.200000,508.000000,71.300000,930.789000,140.987000


In [94]:
#Preparación de datos
data.dropna(inplace=True)
data_encoded = pd.get_dummies(data)
data = data.rename(columns={' FreeStreamVelocity': 'FreeStreamVelocity'})
data_filtered = data.drop(['SuctionSideDisplacementThickness'], axis=1) #Segun resultados, se vera si usara o no.

In [95]:
#Separacion de datos de entrenamiento y prueba
X = data[['AngleOfAttack','Frequency','ChordLength','FreeStreamVelocity','SuctionSideDisplacementThickness']]
y = data['ScaledSoundPressureLevel']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [110]:
#Elaboración de Modelos

# 1.Modelo de Regresión Lineal
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
linear_predictions = linear_model.predict(X_test)


In [111]:
# 2.Modelo de Regresión Lineal con variables Polinomiales
poly_features = PolynomialFeatures(degree=2)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)

poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)
poly_predictions = poly_model.predict(X_test_poly)


In [112]:
#3.Modelo de bosques aleatorios (Random Forest)
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

In [103]:
#Evaluación de todos los modelos
#1. Valor Absoluto
linear_pred = linear_model.predict(X_test)
poly_pred = poly_model.predict(X_test_poly)
rf_pred = rf_model.predict(X_test)

mae_linear = mean_absolute_error(y_test, linear_pred)
mae_poly = mean_absolute_error(y_test, poly_pred)
mae_rf = mean_absolute_error(y_test, rf_pred)

In [104]:
#2. Al cuadrado
mse_linear = mean_squared_error(y_test, linear_pred)
mse_poly = mean_squared_error(y_test, poly_pred)
mse_rf = mean_squared_error(y_test, rf_pred)

In [105]:
#3. Prueba de normalidad
linear_residuals = shapiro(y_test - linear_pred)
poly_residuals = shapiro(y_test - poly_pred)
rf_residuals = shapiro(y_test - rf_pred)

In [113]:
# Calcula las puntuaciones para cada modelo
mae_linear = mean_absolute_error(y_test, linear_predictions)
mse_linear = mean_squared_error(y_test, linear_predictions)

mae_poly = mean_absolute_error(y_test, poly_predictions)
mse_poly = mean_squared_error(y_test, poly_predictions)

mae_rf = mean_absolute_error(y_test, rf_predictions)
mse_rf = mean_squared_error(y_test, rf_predictions)

# Imprime las métricas MAE y MSE de cada modelo
print(f'Modelo de Regresión Lineal:')
print(f'MAE: {mae_linear}')
print(f'MSE: {mse_linear}')
print()

print(f'Modelo de Regresión Lineal con variables Polinomiales:')
print(f'MAE: {mae_poly}')
print(f'MSE: {mse_poly}')
print()

print(f'Modelo de Bosques Aleatorios (Random Forest):')
print(f'MAE: {mae_rf}')
print(f'MSE: {mse_rf}')
print()

# Identifica el modelo con la mejor puntuación basándose en la métrica deseada
best_model_mae = min([mae_linear, mae_poly, mae_rf])
best_model_mse = min([mse_linear, mse_poly, mse_rf])

if best_model_mae == mae_linear:
    best_model = 'Modelo de Regresión Lineal'
elif best_model_mae == mae_poly:
    best_model = 'Modelo de Regresión Lineal con variables Polinomiales'
else:
    best_model = 'Modelo de Bosques Aleatorios (Random Forest)'

print(f'El mejor modelo (según MAE) es: {best_model}')
print(f'El mejor modelo (según MSE) es: {best_model}')

Modelo de Regresión Lineal:
MAE: 4.506975496446938
MSE: 30.55039295745888

Modelo de Regresión Lineal con variables Polinomiales:
MAE: 3.8250219342125993
MSE: 24.12052124456966

Modelo de Bosques Aleatorios (Random Forest):
MAE: 1.539657716186251
MSE: 4.332535985207758

El mejor modelo (según MAE) es: Modelo de Bosques Aleatorios (Random Forest)
El mejor modelo (según MSE) es: Modelo de Bosques Aleatorios (Random Forest)


In [114]:
#El mejor modelo (según MAE) es: Modelo de Bosques Aleatorios (Random Forest)
#El mejor modelo (según MSE) es: Modelo de Bosques Aleatorios (Random Forest)